# Cell Type Assignment with Garnett

Garnett performs hierarchical cell type assignment, information can be found here: https://cole-trapnell-lab.github.io/garnett/docs_m3/

In [1]:
# Get environment variables
marker.list <- Sys.getenv("SNAKEMAKE_MARKER_LIST")
data <- Sys.getenv("SNAKEMAKE_H5AD_INPUT")
output_file <- Sys.getenv("SNAKEMAKE_OUTPUT_FILE")  # Output csv mapping barcodes to cell type data
organsim <- Sys.getenv("SNAKEMAKE_ORGANISM")  # Must be either "human" or "mouse"
# Check that the file exists
if (!file.exists(marker.list)) {
  stop(paste("Marker list file does not exist:", marker.list))
}

if (!file.exists(data)) {
  stop(paste("Input data file does not exist:", data))
}

cat(paste("Marker list file:", marker.list, "\n"))
cat(paste("Input data file:", data, "\n"))
cat(paste("Output file:", output_file, "\n"))
cat(paste("Organism:", organsim, "\n"))

ERROR: Error: Marker list file does not exist: 


In [ ]:
library(garnett)
# Load the gene database according to the organism
if (organsim == "human") {
  gene_db <- require(org.Hs.eg.db)
} else if (organsim == "mouse") {
  gene_db <- require(org.Mm.eg.db)
} else {
  stop("Organism must be either 'human' or 'mouse'.")
}

Load the single cell data as a SingleCellExperiment object and then convert to a cell_data_set (CDS) object

In [ ]:
library(zellkonverter)
setZellkonverterVerbose(TRUE)
adata <- readH5AD(data)
adata

In [ ]:
library(monocle3)
gene_meta = as.data.frame(rowData(adata))
if (!"gene_short_name" %in% colnames(gene_meta)) {
    gene_meta$gene_short_name <- rownames(gene_meta)
}
cds <- new_cell_data_set(
    expression_data = counts(adata),
    cell_metadata = as.data.drame(colData(adata)),
    gene_metadata = gene_meta
)
head(cds)

Evaluate the marker list

In [ ]:
plot_markers(
    cds,
    marker_file = marker.list,
    db=gene_db,
    cds_gene_id_type = "SYMBOL",
    marker_file_gene_id_type = "SYMBOL"
)

Train the Garnett classifier using the marker list

In [ ]:
new_classifier <- train_cell_classifier(
    cds,
    marker_file = marker.list,
    db = gene_db,
    cds_gene_id_type = "SYMBOL",
    marker_file_gene_id_type = "SYMBOL"
)
new_classifier

In [ ]:
feature_genes <- get_feature_genes(new_classifier,
                                   node = "root",
                                   db = gene_db)
feature_genes

Save the classifier to a RDS file

In [ ]:
saveRDS(new_classifier, file = output_file)

Session Information

In [ ]:
sessionInfo()